# MMS-ASR Lab

## Clone fairseq

In [1]:
!git clone https://github.com/pytorch/fairseq

Cloning into 'fairseq'...
remote: Enumerating objects: 34697, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 34697 (delta 73), reused 125 (delta 54), pack-reused 34543
Receiving objects: 100% (34697/34697), 24.78 MiB | 27.73 MiB/s, done.
Resolving deltas: 100% (25188/25188), done.


In [2]:
!mkdir "temp_dir"

%cd "/content/fairseq"
!pip install --editable ./
!pip install tensorboardX

/content/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.7/272.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp310-cp310-linux_x86_64.whl size=9219 sha256=a7416e87a75aae66238cc1c6323cafc178a07fcf82c6e0f3642013981fe105a7
  Stored in directory: /tmp/pip-ephem-wheel-

In [3]:
# %cd "/content"
# !pip install fairseq

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Download MMS model
current model that support Shan's (shn) language is
  - MMS-1B:L1107
  - MMS-1B:all

In [4]:
# go to parent dir
%cd "/content"
!pwd

/content
/content


In [5]:
# # MMS-1B:L1107 - 1107 Languages - MMS-lab Dataset
!wget -P ./asr_models 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_l1107.pt'

# # MMS-1B-all - 1162 Languages - MMS-lab + FLEURS + CV + VP + MLS
# !wget -P ./asr_models 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt'

--2023-05-27 18:12:22--  https://dl.fbaipublicfiles.com/mms/asr/mms1b_l1107.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 52.84.125.65, 52.84.125.71, 52.84.125.2, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|52.84.125.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14094212209 (13G) [binary/octet-stream]
Saving to: ‘./asr_models/mms1b_l1107.pt’

mms1b_l1107.pt      100%[===================>]  13.13G   231MB/s    in 63s     

2023-05-27 18:13:25 (213 MB/s) - ‘./asr_models/mms1b_l1107.pt’ saved [14094212209/14094212209]



## Audio file
audio file requirements
 - .wav
 - sample rate: 16000

 see exaple tts-data from [tts-data](https://github.com/doublehprojects/tts-data/tree/main/audio-data) by haohaaorg

In [6]:
# check current dir
!pwd
%cd "/content"

!wget -P ./audio_samples/ 'https://github.com/doublehprojects/tts-data/blob/main/audio-data/audio2.wav'

# incase audio file is not .wav file or sample rate is not 16000 use ffmpeg to convert it

!ffmpeg -i ./audio_samples/audio2.wav -ar 16000 ./audio_samples/audio_sample.wav

/content
/content
--2023-05-27 18:14:02--  https://github.com/doublehprojects/tts-data/blob/main/audio-data/audio2.wav
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./audio_samples/audio2.wav’

audio2.wav              [ <=>                ] 137.11K  --.-KB/s    in 0.07s   

2023-05-27 18:14:02 (1.81 MB/s) - ‘./audio_samples/audio2.wav’ saved [140402]

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enab

## Transcribe audio

In [7]:
%cd fairseq

/content/fairseq


In [8]:
import os

os.environ["TMPDIR"] = '/content/temp_dir'
os.environ["PYTHONPATH"] = "."
os.environ["PREFIX"] = "INFER"
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["USER"] = "micro"

# model = "/content/asr_models/mms1b_l1107.pt"
# model = "/content/asr_models/mms1b_all.pt"
# lang = "shn"
# audio_sample = "/content/audio_samples/audio1.wav"

!python examples/mms/asr/infer/mms_infer.py --model "/content/asr_models/mms1b_l1107.pt" --lang "shn" --audio "/content/audio_samples/audio_sample.wav"


>>> preparing tmp manifest dir ...
>>> loading model & running inference ...
2023-05-27 18:14:41.457257: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 18:14:43.567629: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Killed
Traceback (most recent call last):
  File "/content/fairseq/examples/mms/asr/infer/mms_infer.py", line 63, in <module>
    process(args)
  File "/content/fairseq/examples/mms/asr/infer/mms_infer.py", line 53, in process
    with open(tmpdir/"hypo.word") as fr:
FileNotFoundError: [Errno 2] No such file or directory: '/content/temp_dir/tmp3mn4ejh1/hypo.word'


# out-of-memory.